## Test load model

In [1]:
import numpy as np
import cv2

# Load an color image in grayscale
img = cv2.imread('/home/thaovt7/workspace/Dataset/Eye_original/maps_bordered/20_1em_20_170120_-100727_00h18m31s_033082.png.png')

In [2]:
np.unique(img)

array([  0,  35,  50,  64,  81,  85, 103, 114, 115, 127, 128, 208, 232,
       244, 255], dtype=uint8)

In [ ]:
model = model_from_json("weights/keras/pspnet101_cityscapes.json")

In [ ]:
from keras.models import load_model

In [ ]:
load_model("weights/keras/pspnet101_cityscapes.h5")

In [ ]:
from keras.utils.generic_utils import CustomObjectScope

In [1]:
import keras

Using TensorFlow backend.


In [4]:
from keras.models import model_from_json

In [5]:
json_path = "weights/keras/pspnet101_cityscapes.json"
h5_path = "weights/keras/pspnet101_cityscapes.h5"

In [6]:
with open(json_path, 'r') as file_handle:
    model = model_from_json(file_handle.read())
model.load_weights(h5_path)

In [8]:
keras.utils.plot_model(model, to_file="model.png", show_shapes=True, show_layer_names=True)

## Test generator

In [1]:
import os
import os
import random
import numpy as np
from scipy.misc import imresize, imread
from scipy.ndimage import zoom
from collections import defaultdict
datadir='/home/thaovt7/workspace/Dataset'
test_nmb =50
separator='_'

In [2]:
def preprocess_img(img, input_shape):
    img = imresize(img, input_shape)
    img = img - DATA_MEAN
    img = img[:, :, ::-1]
    img.astype('float32')
    return img

def update_inputs(batch_size = None, input_size = None, num_classes = None):
  return np.zeros([batch_size, input_size[0], input_size[1], 3]), \
    np.zeros([batch_size, input_size[0], input_size[1], num_classes])

In [4]:
if not os.path.exists(datadir):
    print("ERROR!The folder is not exist")
#listdir = os.listdir(datadir)
data = defaultdict(dict)
# # img from cityscapes
# image_dir = os.path.join(datadir, "imgs/cityscapes")
# image_paths = os.listdir(image_dir)
# for image_path in image_paths:
#     nmb = image_path.split('.')[0]
#     data[nmb]['image'] = image_path

# # image from apollo
# image_dir = os.path.join(datadir, "imgs/apollo")
# image_paths = os.listdir(image_dir)
# for image_path in image_paths:
#     nmb = image_path.split('.')[0]
#     data[nmb]['image'] = image_path

# image from augument
image_dir = os.path.join(datadir, "imgs/annotation")
image_paths = os.listdir(image_dir)
for image_path in image_paths:
    nmb = image_path.split('.')[0]
    data[nmb]['image'] = image_path

# # anno from cityscapes
# anno_dir = os.path.join(datadir, "maps_bordered/cityscapes")
# anno_paths = os.listdir(anno_dir)
# for anno_path in anno_paths:
#     nmb = anno_path.split('.')[0]
#     data[nmb]['anno'] = anno_path

# # anno from apollo
# anno_dir = os.path.join(datadir, "maps_bordered/apollo")
# anno_paths = os.listdir(anno_dir)
# for anno_path in anno_paths:
#     nmb = anno_path.split('.')[0]
#     data[nmb]['anno'] = anno_path

# anno from augument
anno_dir = os.path.join(datadir, "maps_bordered/annotation")
anno_paths = os.listdir(anno_dir)
for anno_path in anno_paths:
    nmb = anno_path.split('.')[0]
    data[nmb]['anno'] = anno_path


values = list(data.values())
random.shuffle(values)
print(len(values))
test_nmb = int(len(values)/10)
#print(test_nmb)

61965


In [6]:
batch_size =1
input_size = (713,713)
nb_classes = 5
while 1:
    random.shuffle(values)
    images, labels = update_inputs(batch_size=batch_size,
       input_size=input_size, num_classes=nb_classes)
    for i, d in list(enumerate(values))[:1]:
      #print(d['image'].split('_')[0])
        if d['image'].split('_')[0] == 'a':
            image_dir = os.path.join(datadir, "imgs/apollo")
            anno_dir = os.path.join(datadir, "maps_bordered/apollo")
        elif d['image'].split('_')[0] == '20':
            image_dir = os.path.join(datadir, "imgs/annotation")
            anno_dir = os.path.join(datadir, "maps_bordered/annotation")
        else:
            image_dir = os.path.join(datadir, "imgs/cityscapes")
            anno_dir = os.path.join(datadir, "maps_bordered/cityscapes")
        #   #print(y)
    #   #print(d['anno'])
    #   #print("Shape of y is {}".format(y.shape))
        print(os.path.join(anno_dir, d['anno']))
        img = imresize(imread(os.path.join(image_dir, d['image']), mode='RGB'), input_size)
        y = imread(os.path.join(anno_dir, d['anno']), mode='L')
        h, w = input_size
        #   #print(y)
        y = zoom(y, (1.*h/y.shape[0], 1.*w/y.shape[1]), order=0, mode='nearest', prefilter=False)
        print(np.unique(y))

        # config class
        y[y == 0] = 4 # void, trainID = 4
        y[y == 90] = 0 #road, trainID = 0
        y[y == 119] = 1 #sidewalk, trainID = 1
        y[y == 33] = 2 #cub, trainID = 2
        y[y == 81] = 3 #mycar, trainID = 3
        print (np.unique(y))
    # for i in range(y.shape[0]):
    #     for j in range(y.shape[1]):
    #         if y[i][j] == 90:
    #             y[i][j] = 0
    #         elif y[i][j] == 119:
    #             y[i][j] = 1
    #         else:
    #             y[i][j] = 2
      #print("Shape of y is {}".format(y.shape))
      #print(np.arange(nb_classes) == y[:,:,None])
      #print(y[:,:,None])
    #   y = (np.arange(nb_classes) == y[:,:,None]).astype('float32')
    #   #print("Shape of y is {}".format(y.shape))
    #   #print(y[3])
    #   assert y.shape[2] == nb_classes
    #   images[i % batch_size] = img
    #   labels[i % batch_size] = y

/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_21_206th_mah04082_00h10m04s_018104.png.png


/home/thaovt7/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/thaovt7/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/thaovt7/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_15_202id_mah04071_00h09m17s_016697.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_2_204ty_mah00135_00h39m47s_071540.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_11_202id_mah04071_00h28m28s_051201.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_3_206th_mah04082_00h11m04s_019923.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_25_205yy_mah04081_00h23m19s_041941.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_18_205yy_mah04081_00h41m05s_073905.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_11_204ty_mah00135_00h38m37s_069464.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/

/home/thaovt7/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_28_202id_mah04071_00h46m08s_082986.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_17_204ty_mah00135_00h44m40s_080323.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_15_202id_mah04071_00h18m14s_032816.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_17_204ty_mah00135_00h09m06s_016370.png.png
[ 0 33 90]
[0 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_21_205yy_mah04081_00h21m54s_039409.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_16_204ty_mah00135_00h50m23s_090622.png.png
[  0  33  90 119]
[0 1 2 4]
/home/thaovt7/workspace/Dataset/maps_bordered/annotation/20_24_202id_mah04071_00h35m53s_064536.png.png


KeyboardInterrupt: 

In [109]:
y = (np.arange(5) == y[:,:,None]).astype('float32')

In [114]:
y[1:100][1:100][4]

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [72]:
y[500][300:500]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0.

In [32]:
np.arange(5).shape

(5,)

In [161]:
random.shuffle(values)

In [162]:
values

[{'anno': 'aachen_000107_000019_gtFine_labelIds.png',
  'image': 'aachen_000104_000019_leftImg8bit.png'}]

In [153]:
values[0]['anno']

'aachen_000107_000019_gtFine_labelIds.png'

In [55]:
for image_path in image_paths:
    nmb = image_path.split(separator)[0]
    data[nmb]['image'] = image_path
    print (data[nmb]['image'])

aachen_000149_000019_leftImg8bit.png
aachen_000137_000019_leftImg8bit.png
aachen_000147_000019_leftImg8bit.png
aachen_000026_000019_leftImg8bit.png
aachen_000068_000019_leftImg8bit.png
aachen_000041_000019_leftImg8bit.png
aachen_000050_000019_leftImg8bit.png
aachen_000145_000019_leftImg8bit.png
aachen_000047_000019_leftImg8bit.png
aachen_000004_000019_leftImg8bit.png
aachen_000021_000019_leftImg8bit.png
aachen_000098_000019_leftImg8bit.png
aachen_000040_000019_leftImg8bit.png
aachen_000059_000019_leftImg8bit.png
aachen_000017_000019_leftImg8bit.png
aachen_000130_000019_leftImg8bit.png
aachen_000099_000019_leftImg8bit.png
aachen_000072_000019_leftImg8bit.png
aachen_000007_000019_leftImg8bit.png
aachen_000121_000019_leftImg8bit.png
aachen_000012_000019_leftImg8bit.png
aachen_000144_000019_leftImg8bit.png
aachen_000039_000019_leftImg8bit.png
aachen_000131_000019_leftImg8bit.png
aachen_000151_000019_leftImg8bit.png
aachen_000055_000019_leftImg8bit.png
aachen_000034_000019_leftImg8bit.png
a

## Test config GPU

In [1]:
import tensorflow as tf

In [2]:
sess = tf.Session()

In [3]:
hello_world = tf.constant("Hello, world!")

In [5]:
print (sess.run(hello_world))

b'Hello, world!'


In [7]:
print (sess.run(tf.constant(123)*tf.constant(456)))

56088


In [12]:
with tf.device('/gpu:0'):

    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')

    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')

    c = tf.matmul(a, b)

# Creates a session with log_device_placement set to True.

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Runs the op.

print (sess.run)

<bound method BaseSession.run of <tensorflow.python.client.session.Session object at 0x7f62e9ad4c88>>


In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/home/thaovt7/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6576996484482223453
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 91291648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8394015821686840336
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
